In [ ]:
import sys
sys.path = ["/home/jmetzen/Repositories/scikit-learn", 
            "/home/jmetzen/Repositories/team/src/bayesian_optimization"] + sys.path

import numpy as np
from scipy.stats import multivariate_normal, entropy
from scipy.linalg import eigvals

import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

paper_path = "/home/jmetzen/Repositories/BesMan/SVN/Documentation/Publications/PeerReviewed/drafts/2016_ICML/pics/"

plt.rcParams.update({'axes.labelsize': 11,
                     'axes.titlesize': 12,
                     'text.fontsize': 11,
                     'xtick.labelsize': 8,
                     'ytick.labelsize': 8,
                     'legend.fontsize': 7,
                     'legend.fancybox': True,
                     'font.family': 'serif'})

In [ ]:
x_values = np.linspace(0.0, 1.0-0.001, 11)
y_values = np.linspace(0.0, 1.0-0.001, 11)
X, Y = np.meshgrid(x_values, y_values)
Z_es = np.empty_like(X)
Z_mrs = np.empty_like(X)
for i, x in enumerate(x_values):
    for j, y in enumerate(y_values):
        mean = np.array([1, y])
        cov = np.array([[1.0, x], [x, 1.0]])
        
        distr = multivariate_normal(mean=mean, cov=cov)
        samples = distr.rvs(10000)
        Z_es[i, j] = entropy(np.bincount(samples.argmax(1)))
        Z_mrs[i, j] = np.mean(samples.max(1) - samples[:, mean.argmax()])

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.title("Entropy")
plt.pcolor(X, Y, Z_es.T)
plt.xlabel("x")
plt.ylabel("y")
plt.colorbar()

plt.subplot(1, 3, 2)
plt.title("Expected Regret")
plt.pcolor(X, Y, Z_mrs.T)
plt.xlabel("x")
plt.yticks([])
plt.colorbar()

plt.tight_layout()
# Add parameterized distribution

In [ ]:
from sklearn.gaussian_process.kernels import RBF

from bayesian_optimization import GaussianProcessModel, MinimalRegretSearch, EntropySearch, ExpectedImprovement

X = np.array([[0, 1.25, 1.75, 3]]).T
y = np.array([0.0, 1.0, 1.0, 0.0])
boundaries = np.array([[0, 4]])

 
kernel = RBF(length_scale=0.75, length_scale_bounds="fixed")

model = GaussianProcessModel(kernel=kernel, alpha=1e-10)
model.fit(X, y)

n_gp_samples = 1000

# Generate candidate points that can be reused later
mrs = MinimalRegretSearch(model=model, n_gp_samples=n_gp_samples, n_candidates=41, n_trial_points=250,
                          n_samples_y=51, rng_seed=i)
mrs.set_boundaries(boundaries)
X_candidate = mrs.X_candidate

In [ ]:
fig_width = 234.8775 / 72.27 # Get fig_width_pt from LaTeX using \showthe\columnwidth
fig_height = fig_width * 0.66  # height in inches
plt.figure(0, dpi=400, figsize=(fig_width, fig_height))

X_plot = np.linspace(0, 4, 81)
y_mean, y_std = model.gp.predict(X_plot[:, np.newaxis], return_std=True)
plt.plot(X_plot, y_mean, c='k', label="GP posterior")
plt.scatter(X[:, 0], y, c='g', zorder=10, s=20)
plt.fill_between(X_plot, y_mean - y_std, y_mean + y_std, alpha=0.3, color='k')
for i in range(1):
    mrs = MinimalRegretSearch(model=model, n_gp_samples=n_gp_samples,
                              n_samples_y=51, rng_seed=i)
    mrs.set_boundaries(boundaries, X_candidate=X_candidate)
    mrs_point = MinimalRegretSearch(model=model, n_gp_samples=n_gp_samples,
                                    n_samples_y=51, point=True, rng_seed=i)
    mrs_point.set_boundaries(boundaries, X_candidate=X_candidate)
    es = EntropySearch(model=model, n_gp_samples=n_gp_samples, 
                       n_samples_y=51, rng_seed=i)
    es.set_boundaries(boundaries, X_candidate=X_candidate)
    ei = ExpectedImprovement(model=model)
    ei.set_boundaries(boundaries)

    X_plot = np.linspace(0, 4, 81)
    y_mean, y_std = model.gp.predict(X_plot[:, np.newaxis], return_std=True)
    y_mrs = mrs(X_plot[:, np.newaxis])
    y_mrs_point = mrs_point(X_plot[:, np.newaxis])
    y_es = es(X_plot[:, np.newaxis])
    y_ei = ei(X_plot[:, np.newaxis], incumbent=y.max())
    
    plt.plot(X_plot, y_es / (2 * y_es.mean()), c='b', alpha=1.0, 
             label="ES" if i == 0 else None)
    plt.plot(X_plot, y_mrs / (2 * y_mrs.mean()), c='r', alpha=1.0, 
             label="MRS" if i == 0 else None)
    plt.plot(X_plot, y_mrs_point / (2 * y_mrs_point.mean()), c='c', alpha=1.0, 
             label=r"MRS$^{point}$" if i == 0 else None)
    plt.plot(X_plot, y_ei / (5 * y_ei.mean()), c='m',
             label="EI" if i == 0 else None)
    
plt.xlim(0, 4)
plt.ylim(-1.0, 1.5)
plt.xlabel(r"$\mathbf{x}^q$")
plt.legend(loc="best", ncol=3)
plt.tight_layout()
plt.savefig(paper_path + "acq_comparison.png", dpi=400)

In [ ]:
X_candidate = np.linspace(0, 4, 41)[:, np.newaxis]
mrs = MinimalRegretSearch(model=model, n_gp_samples=n_gp_samples,
                          n_samples_y=51, rng_seed=i)
mrs.set_boundaries(boundaries, X_candidate=X_candidate)

f_mean, f_cov = \
    mrs.model.gp.predict(mrs.X_candidate, return_cov=True)
f_samples = np.random.multivariate_normal(f_mean, f_cov,
                                          mrs.n_gp_samples*10).T
bincount = \
    np.bincount(np.argmax(f_samples, 0), minlength=f_mean.shape[0])
p_max = bincount / float(mrs.n_gp_samples * 10)
regret = (f_samples.max(0) - f_samples).mean(1)

opt_index = f_mean.argmax()
mean_regret = np.maximum(0, f_samples - f_samples[opt_index]).mean(1)
bincount_2 = \
    np.bincount(np.argmax(f_samples, 0), minlength=f_mean.shape[0])
bincount_2[opt_index] = 0
regret_counter = bincount_2 / float(sum(bincount_2))

In [ ]:
fig_width = 234.8775 / 72.27  # Get fig_width_pt from LaTeX using \showthe\columnwidth
fig_height = fig_width * 0.66  # height in inches
fig = plt.figure(0, dpi=400, figsize=(fig_width, fig_height))

ax = fig.add_subplot(111)
ax.plot(X_plot, y_mean, c='k', label="GP posterior")
ax.scatter(X[:, 0], y, c='g', zorder=10, s=20)
ax.fill_between(X_plot, y_mean - y_std, y_mean + y_std, alpha=0.3, color='k')
ax.plot(mrs.X_candidate, regret, c='r', label=r"ER($\mathbf{x}$)")
ax.plot(mrs.X_candidate, p_max, c='b', label="$p^\star$")
#ax.plot(mrs.X_candidate, regret * p_max, c='m', label=r"ER($\theta$) * $p_{max}$")
#ax.plot(mrs.X_candidate, regret_counter, c='c', label=r"$p(f(\theta) > f(1.5))$")
ax.set_xlim(0, 4)
ax.set_ylim(-0.8, 1.25)
ax.set_xlabel(r"$\mathbf{x}$")
ax.grid("off")
ax.legend(loc='lower left')

ax2 = ax.twinx()
ax2.plot(mrs.X_candidate, mean_regret, c='m', label=r"$\mathbb{E}[$max$(f(\mathbf{x}) - f(1.5), 0)]$")
ax.set_xlim(0, 4)
ax2.set_ylim(-0.035, 0.054)
ax2.grid("off")
ax2.legend(loc="lower right")
plt.tight_layout()
plt.savefig(paper_path + "regret_illustration.png", dpi=400)